Data Preparation

In [7]:
# Data Reading
from pandas import read_csv, get_dummies, DataFrame, Series
bank_data=read_csv('/content/drive/MyDrive/Colab Notebooks/Applied Analytics & Machine Learning/CA 1/bank.csv')
print(bank_data.shape)

# Data Encoding
bank_data['default']= bank_data['default'].map({'yes':0,'no':1})
bank_data['housing']= bank_data['housing'].map({'yes':0,'no':1})
bank_data['loan']= bank_data['loan'].map({'yes':0,'no':1})
bank_data['y']= bank_data['y'].map({'yes':0,'no':1})
bank_data['poutcome']= bank_data['poutcome'].map({'unknown':0,'other':1,'failure':2,'success':3})
bank_data['contact']= bank_data['contact'].map({'unknown':0,'telephone':1,'cellular':2})
bank_data['month']= bank_data['month'].map({'jan':0,'feb':1,'mar':2,'apr':3,'may':4,'jun':5,'jul':6,'aug':7,'sep':8,'oct':9,'nov':10,'dec':11})
bank_data=get_dummies(bank_data,['job','education','marital'],dtype=int)
print(bank_data.info())
# Dividing data into x and y
x=bank_data.drop('y',axis=1)
y=bank_data['y']
print(x.shape)
print(y.shape)

# Data Scaling of x
from sklearn.preprocessing import StandardScaler
x_scaled=StandardScaler().fit_transform(x)
DataFrame(x_scaled)

#Data spliting
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x_scaled,y,test_size=0.3, random_state=100)
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

# Data Balancing
from imblearn.over_sampling import SMOTE
x_train,y_train=SMOTE(random_state=100).fit_resample(x_train,y_train)

(4521, 17)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4521 entries, 0 to 4520
Data columns (total 33 columns):
 #   Column              Non-Null Count  Dtype
---  ------              --------------  -----
 0   age                 4521 non-null   int64
 1   default             4521 non-null   int64
 2   balance             4521 non-null   int64
 3   housing             4521 non-null   int64
 4   loan                4521 non-null   int64
 5   contact             4521 non-null   int64
 6   day                 4521 non-null   int64
 7   month               4521 non-null   int64
 8   duration            4521 non-null   int64
 9   campaign            4521 non-null   int64
 10  pdays               4521 non-null   int64
 11  previous            4521 non-null   int64
 12  poutcome            4521 non-null   int64
 13  y                   4521 non-null   int64
 14  job_admin.          4521 non-null   int64
 15  job_blue-collar     4521 non-null   int64
 16  job_entrepreneur    4521 non-nu

In [8]:
# Using pipeline (method #3)
from imblearn.pipeline import Pipeline
from sklearn import ensemble
from sklearn.model_selection import GridSearchCV
RF_classifier = Pipeline([('balancing', SMOTE(random_state = 100)),('classification', ensemble.RandomForestClassifier(criterion='entropy', max_features='sqrt', random_state=1))]) # building classifier
no_trees = {'classification__n_estimators': [100,150,200,250,300,350,400,450,500]}
grid_search = GridSearchCV(estimator=RF_classifier, param_grid=no_trees, scoring='recall', cv=5)
grid_search.fit(x_scaled, y)

best_parameters = grid_search.best_params_
print(best_parameters)
best_result = grid_search.best_score_
print(best_result)

{'classification__n_estimators': 250}
0.9484999999999999


In [14]:
# Building random forest (method #1 ) with the best number of trees
RF_classifier = ensemble.RandomForestClassifier(n_estimators=250, criterion='entropy', max_features='sqrt', random_state=10)
RF_classifier.fit(x_train,y_train)
y_pred=RF_classifier.predict(x_test)# testing
imp_features = Series(RF_classifier.feature_importances_, index=list(x)).sort_values(ascending=False)

In [16]:
# calculating accuaracy
from sklearn import metrics
accuracy=metrics.accuracy_score(y_test, y_pred) # calculating accuaracy
print("Accuracy = ", accuracy) # Is this a good metric??
con_matrix = metrics.confusion_matrix(y_test, y_pred)
print (con_matrix)
recall = metrics.recall_score(y_test, y_pred)
print ('Recall = ',recall)
percision=metrics.precision_score(y_test, y_pred)
print('Percision = ',percision)
f1=metrics.f1_score(y_test, y_pred)
print('f1 = ',f1)

Accuracy =  0.8975681650700074
[[  73   69]
 [  70 1145]]
Recall =  0.9423868312757202
Percision =  0.943163097199341
f1 =  0.9427748044462742


In [11]:
#  Support Vector Classifier (method#3)
from sklearn import svm
from imblearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
SV_classifier2 = Pipeline([('balancing', SMOTE(random_state = 101)),('classification', svm.SVC() ) ]) # bulilding classifier
kernels_c = {'classification__kernel': ['linear','poly','rbf','sigmoid'], 'classification__C': [.001,.01,.1,1,10,100]}
grid_search1 = GridSearchCV(estimator=SV_classifier2, param_grid=kernels_c, scoring='recall', cv=5)
grid_search1.fit(x_scaled, y)

best_parameters = grid_search1.best_params_
print(best_parameters)
best_result = grid_search1.best_score_
print(best_result)

{'classification__C': 0.001, 'classification__kernel': 'poly'}
0.9834999999999999


In [12]:
from sklearn import svm
SV_classifier = svm.SVC(kernel='poly', C = 0.001)  # building classifier
SV_classifier.fit(x_train, y_train) # training
y_predect= SV_classifier.predict(x_test)  #testing

In [13]:
# Acuracy and confusion matrix
from sklearn import metrics
Accuracy=metrics.accuracy_score(y_test, y_predect) # calculating accuaracy
print('Accuracy: ', Accuracy) # Is this a good metric??
con_matrix = metrics.confusion_matrix(y_test, y_pred)
print (con_matrix)
recall = metrics.recall_score(y_test, y_predect)
print ('recall', recall)
percision=metrics.precision_score(y_test, y_predect)
print('percision', percision)
f1=metrics.f1_score(y_test, y_predect)
print('f1 = ',f1)

Accuracy:  0.8931466470154753
[[  72   70]
 [  72 1143]]
recall 0.9950617283950617
percision 0.8968842729970327
f1 =  0.9434256730394069
